In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis
from delierium.MatrixOrder import Context
from delierium.helpers import latexer
from sage.misc.html import html
from IPython.core.debugger import set_trace


In [2]:
x = var('x')
y=function('y')

In [3]:
ode=4*diff(y(x),x,x)*y(x) - 3* (diff(y(x),x)**2)-12*(y(x)**3)

In [4]:
p=prolongationODE(ode, y, x)

In [5]:
for _p in p:
    show(latexer(_p))

\(-4 y  \xi_{yy} y_{x}^3 + 4 y  \phi_{yy} y_{x}^2 - 8 y  \xi_{yx} y_{x}^2 + 6  \xi_{y} y_{x}^3 - 12 y  \xi_{y} y_{x} y_{x x} - 36 \phi(y, x) y^2 + 8 y  \phi_{yx} y_{x} - 4 y  \xi_{xx} y_{x} - 6  \phi_{y} y_{x}^2 + 6  \xi_{x} y_{x}^2 + 4 y  \phi_{y} y_{x x} - 8 y  \xi_{x} y_{x x} + 4 y  \phi_{xx} - 6  \phi_{x} y_{x} + 4 \phi(y, x) y_{x x}\)

In [6]:
def infinitesimalsODE (ode, dependent, independent):
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    s1  = solve(ode==0, diff(dependent(independent),independent,independent))
    ode1= prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify()
    #print((ode1))
    # so far it is checked manually and with mathematica
    l = [_ [0] for _ in ode1.coefficients(diff(dependent(independent), independent))]
    # remove hardcoded xi, phi
    return l

In [7]:
ode=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode, y ,x)

In [8]:
for _ in inf:
    show(latexer(_))

\(12 y^3  \phi_{y} - 24 y^3  \xi_{x} - 24 \phi(y, x) y^2 + 4 y  \phi_{xx}\)

\(-36 y^3  \xi_{y} + 8 y  \phi_{yx} - 4 y  \xi_{xx} - 6  \phi_{x}\)

\(4 y  \phi_{yy} - 8 y  \xi_{yx} + 3 \phi(y, x)/y - 3  \phi_{y}\)

\(-4 y  \xi_{yy} - 3  \xi_{y}\)

In [9]:
ode = diff(y(x),x,x)+3*y(x)*diff(y(x),x)+y(x)**3
inf = infinitesimalsODE(ode, y ,x)
for _ in inf:
    show(latexer(_))

\(-y^3  \phi_{y} + 2 y^3  \xi_{x} + 3 \phi(y, x) y^2 + 3 y  \phi_{x} +  \phi_{xx}\)

\(3 y^3  \xi_{y} + 3 y  \xi_{x} + 3 \phi(y, x) + 2  \phi_{yx} -  \xi_{xx}\)

\(6 y  \xi_{y} +  \phi_{yy} - 2  \xi_{yx}\)

\(- \xi_{yy}\)

In [10]:
ode=diff(y(x),x,x)

In [11]:
inf = infinitesimalsODE(ode, y ,x)
for _ in inf:
    show(latexer(_))

\( \phi_{xx}\)

\(2  \phi_{yx} -  \xi_{xx}\)

\( \phi_{yy} - 2  \xi_{yx}\)

\(- \xi_{yy}\)

In [15]:
from delierium.helpers import latexer
x,t = var('x t')
u=function('u')(x,t)
v=function('v')(x,t)
ctx = Context([u,v],[x,t])
jb = Janet_Basis([diff(v, t) - (u**2 +1)*diff(u, x)-u, diff(u,t)-diff(v,x)],[u,v],[x,t])
for _ in jb.S:
    show(latexer(_.expression()))

\(u_{t} - v_{x}\)

\(u/(u^2 + 1) - v_{t}/(u^2 + 1) + u_{x}\)

\(-2 u^2 v_{x}/(u^4 + 2 u^2 + 1) + 2 u/(u^4 + 2 u^2 + 1) - v_{t t}/(u^2 + 1) + v_{x x}\)